In [ ]:
from dataloader.result_loader import KinectResultLoader, ArbeResultLoader, OptitrackResultLoader
import numpy as np
import os
root_path = "/home/nesc525/chen/3DSVC/__test__/default"

arbe_loader = ArbeResultLoader(root_path)
arbe_loader.file_dict['arbe'] = arbe_loader.file_dict['arbe'].drop_duplicates(subset=['dt'],keep='first')
ids = arbe_loader.file_dict['arbe']['id']
ids = sorted(map(int, ids))
print(ids)
arbe_loader[ids[0]]["arbe"]
os.listdir(root_path)
frames = range(0, 5, 1)
print(frames)
ids[frames]

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from message.dingtalk import TimerBot

bot =TimerBot()
fig = plt.figure()
fig.add_subplot(1, 1, 1).plot([1,2,3], [1,2,3])
fig.canvas.draw()
img = cv2.cvtColor(np.asarray(fig.canvas.buffer_rgba()), cv2.COLOR_RGBA2BGR)
bot.add_md("test", "【TEST】 \n ![img]({})".format(bot.img2b64(img)))
bot.enable()

In [2]:
def generator():
    for i in range(10):
        yield i
    return 1

for i in generator():
    print(i)
print(generator())


0
1
2
3
4
5
6
7
8
9
<generator object generator at 0x7f92e15b7a50>
